In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from pathlib import Path
import os

In [5]:
# Kaggle Hub setup
import kagglehub
from kagglehub import KaggleDatasetAdapter

In [6]:
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [7]:
data_path="C:/Users/russe/OneDrive/Desktop/Portfolio/Next AAA Title"

In [8]:
def load_imdb_data(data_path):
    """
    Load IMDb data with memory optimization and business focus
    """
    
    print("📊 Loading IMDb data with memory optimization...")
    
    # Load movies with minimal columns for initial filtering
    print("  🎬 Loading movies (basic info only)...")
    movies_basic = pd.read_csv(
        f'{data_path}/Data/title.basics.tsv',
        sep='\t',
        na_values='\\N',
        usecols=['tconst', 'titleType', 'primaryTitle', 'startYear', 'genres', 'runtimeMinutes']
    )
    
    # Filter for movies 2010+ only (business relevance + memory efficiency)
    movies = movies_basic[
        (movies_basic['titleType'] == 'movie') & 
        (movies_basic['startYear'] >= 2010)
    ].copy()
    
    print(f"    ✅ Filtered to {len(movies):,} recent movies (2010+)")
    
    # Load ratings
    print("  ⭐ Loading ratings...")
    ratings = pd.read_csv(
        f'{data_path}/Data/title.ratings.tsv',
        sep='\t'
    )
    
    # Merge and filter for movies with decent vote counts
    movies_with_ratings = movies.merge(ratings, on='tconst', how='inner')
    
    # Filter for movies with at least 50 votes (quality threshold)
    movies_with_ratings = movies_with_ratings[movies_with_ratings['numVotes'] >= 50].copy()
    
    print(f"    ✅ Final dataset: {len(movies_with_ratings):,} movies with quality ratings")
    
    return movies, ratings, movies_with_ratings

In [9]:
def load_actors(data_path, movies_df):
    """
    Load actor data efficiently for 2020 analysis
    """
    
    print("  👥 Loading actor data (targeted)...")
    
    # Get list of movie IDs for 2020 to reduce data loading
    target_years = [2017, 2018, 2019, 2020, 2021]  # Expand slightly for better sample
    target_movies = movies_df[movies_df['startYear'] >= 2010]['tconst'].unique()
    
    print(f"    Target movies for actor analysis: {len(target_movies):,}")
    
    # Load principals with filtering
    chunk_size = 1000000
    principals_filtered = []
    
    print("    Loading principals in chunks...")
    for chunk in pd.read_csv(
        f'{data_path}/Data/title.principals.tsv',
        sep='\t',
        na_values='\\N',
        chunksize=chunk_size,
        usecols=['tconst', 'nconst', 'category', 'ordering']
    ):
        # Filter for target movies and actors only
        filtered_chunk = chunk[
            (chunk['tconst'].isin(target_movies)) &
            (chunk['category'].isin(['actor', 'actress']))
        ]
        
        if len(filtered_chunk) > 0:
            principals_filtered.append(filtered_chunk)
    
    if principals_filtered:
        principals = pd.concat(principals_filtered, ignore_index=True)
        print(f"    ✅ Actor connections loaded: {len(principals):,}")
    else:
        print("    ⚠️ No actor data found for target years")
        principals = pd.DataFrame()
    
    # Load names for actors in our dataset
    if len(principals) > 0:
        target_actors = principals['nconst'].unique()
        
        print(f"    Loading names for {len(target_actors):,} actors...")
        
        # Read names in chunks and filter
        names_filtered = []
        for chunk in pd.read_csv(
            f'{data_path}/Data/name.basics.tsv',
            sep='\t',
            na_values='\\N',
            chunksize=500000,
            usecols=['nconst', 'primaryName']
        ):
            filtered_chunk = chunk[chunk['nconst'].isin(target_actors)]
            if len(filtered_chunk) > 0:
                names_filtered.append(filtered_chunk)
        
        if names_filtered:
            names = pd.concat(names_filtered, ignore_index=True)
            print(f"    ✅ Actor names loaded: {len(names):,}")
        else:
            names = pd.DataFrame()
    else:
        names = pd.DataFrame()
    
    return principals, names

In [10]:
movies_df, ratings_df, movies_ratings_df = load_imdb_data(data_path)
principals_df, names_df = load_actors(data_path, movies_df)

📊 Loading IMDb data with memory optimization...
  🎬 Loading movies (basic info only)...
    ✅ Filtered to 282,615 recent movies (2010+)
  ⭐ Loading ratings...
    ✅ Final dataset: 83,583 movies with quality ratings
  👥 Loading actor data (targeted)...
    Target movies for actor analysis: 282,615
    Loading principals in chunks...
    ✅ Actor connections loaded: 1,516,184
    Loading names for 749,122 actors...
    ✅ Actor names loaded: 748,836


In [8]:
movies_df.to_csv("movies.csv")
ratings_df.to_csv("ratings.csv")
movies_ratings_df.to_csv("movies_ratings.csv")
principals_df.to_csv("principals.csv")
names_df.to_csv("names.csv")

In [11]:
def data_information(movies, ratings, names, principals):
    """
    Comprehensive data quality assessment for business decision-making
    """
    
    print(f"\n" + "="*60)
    print("🔍 COMPREHENSIVE DATA QUALITY ASSESSMENT")
    print("="*60)
    
    # Create primary analysis dataset
    movies_with_ratings = movies.merge(ratings, on='tconst', how='inner')
    print(f"\n📊 Primary Dataset Created:")
    print(f"  Movies with ratings: {len(movies_with_ratings):,}")
    print(f"  Coverage: {len(movies_with_ratings)/len(movies)*100:.1f}% of movies have ratings")
    
    # Temporal coverage analysis
    print(f"\n📅 Temporal Coverage Analysis:")
    year_stats = movies['startYear'].describe()
    print(f"  Year range: {year_stats['min']:.0f} - {year_stats['max']:.0f}")
    print(f"  Movies per decade:")
    
    # Movies by decade
    decade_counts = movies[movies['startYear'] >= 1980].copy()
    decade_counts['decade'] = (decade_counts['startYear'] // 10) * 10
    decade_summary = decade_counts.groupby('decade').size()
    
    for decade, count in decade_summary.tail(5).items():
        print(f"    {int(decade)}s: {count:,} movies")
    
    # Focus on recent movies (business relevance)
    recent_movies = movies_with_ratings[movies_with_ratings['startYear'] >= 2015]
    print(f"\n🎯 Business-Relevant Dataset (2015+):")
    print(f"  Recent movies: {len(recent_movies):,} ({len(recent_movies)/len(movies_with_ratings)*100:.1f}%)")
    
    # Rating quality analysis
    print(f"\n⭐ Rating Quality Metrics:")
    print(f"  Average rating: {movies_with_ratings['averageRating'].mean():.2f}/10")
    print(f"  Rating standard deviation: {movies_with_ratings['averageRating'].std():.2f}")
    print(f"  Median votes: {movies_with_ratings['numVotes'].median():,.0f}")
    
    # Vote distribution analysis
    vote_thresholds = [100, 1000, 10000, 100000]
    print(f"\n  Vote credibility analysis:")
    for threshold in vote_thresholds:
        count = len(movies_with_ratings[movies_with_ratings['numVotes'] >= threshold])
        pct = count / len(movies_with_ratings) * 100
        print(f"    {threshold:,}+ votes: {count:,} movies ({pct:.1f}%)")
    
    
    # Genre analysis
    print(f"\n🎭 Genre Coverage Analysis:")
    all_genres = []
    for genres in movies['genres'].dropna():
        all_genres.extend([g.strip() for g in genres.split(',')])
    
    genre_counts = pd.Series(all_genres).value_counts()
    print(f"  Total unique genres: {len(genre_counts)}")
    print(f"  Top 5 genres:")
    for genre, count in genre_counts.head(5).items():
        print(f"    {genre}: {count:,} movies")
    
    
    return movies_with_ratings, recent_movies, genre_counts


In [12]:
movies_with_ratings, recent_movies, genre_counts = data_information(
    movies_df, ratings_df, names_df, principals_df
)


🔍 COMPREHENSIVE DATA QUALITY ASSESSMENT

📊 Primary Dataset Created:
  Movies with ratings: 152,447
  Coverage: 53.9% of movies have ratings

📅 Temporal Coverage Analysis:
  Year range: 2010 - 2032
  Movies per decade:
    2010s: 168,786 movies
    2020s: 113,823 movies
    2030s: 6 movies

🎯 Business-Relevant Dataset (2015+):
  Recent movies: 110,282 (72.3%)

⭐ Rating Quality Metrics:
  Average rating: 6.27/10
  Rating standard deviation: 1.52
  Median votes: 64

  Vote credibility analysis:
    100+ votes: 65,190 movies (42.8%)
    1,000+ votes: 22,331 movies (14.6%)
    10,000+ votes: 5,539 movies (3.6%)
    100,000+ votes: 1,186 movies (0.8%)

🎭 Genre Coverage Analysis:
  Total unique genres: 26
  Top 5 genres:
    Drama: 96,581 movies
    Documentary: 88,425 movies
    Comedy: 45,882 movies
    Thriller: 22,440 movies
    Horror: 21,326 movies


In [19]:
def create_business_ready_dataset(movies_with_ratings):
    """
    Create analysis-ready dataset with business-relevant features
    """
    
    print(f"\n🎯 Creating Business-Ready Dataset...")
    
    # Focus on movies with sufficient data for analysis
    business_dataset = movies_with_ratings.copy()
    
    # Add derived business metrics
    print("  📊 Adding business metrics...")
    
    # Weighted rating (IMDb formula)
    m = 1000  # minimum votes threshold
    C = business_dataset['averageRating'].mean()
    
    business_dataset['weighted_rating'] = (
        (business_dataset['numVotes'] / (business_dataset['numVotes'] + m)) * business_dataset['averageRating'] +
        (m / (business_dataset['numVotes'] + m)) * C
    )
    
    # Credibility flag
    business_dataset['high_credibility'] = business_dataset['numVotes'] >= 1000
    
    # Recent movie flag
    business_dataset['recent_movie'] = business_dataset['startYear'] >= 2015
    
    # Genre processing
    business_dataset['genre_count'] = business_dataset['genres'].str.count(',') + 1
    business_dataset['genre_count'] = business_dataset['genre_count'].fillna(0)

    business_dataset['runtimeMinutes'] = pd.to_numeric(business_dataset['runtimeMinutes'], errors='coerce')
    business_dataset['runtimeMinutes_filled'] = business_dataset['runtimeMinutes'].fillna(120)  # Default 2 hours
    
    # Runtime categories
    business_dataset['runtime_category'] = pd.cut(
        business_dataset['runtimeMinutes'],
        bins=[0, 90, 120, 150, 300],
        labels=['Short', 'Standard', 'Long', 'Very Long'],
        include_lowest=True
    )
    
    print(f"  ✅ Business dataset ready: {len(business_dataset):,} movies")
    print(f"    High credibility movies: {business_dataset['high_credibility'].sum():,}")
    print(f"    Recent movies (2015+): {business_dataset['recent_movie'].sum():,}")
    
    return business_dataset

In [20]:
business_ready_data = create_business_ready_dataset(movies_with_ratings)


🎯 Creating Business-Ready Dataset...
  📊 Adding business metrics...
  ✅ Business dataset ready: 152,447 movies
    High credibility movies: 22,331
    Recent movies (2015+): 110,282


In [ ]:
def generate_data_summary(business_dataset, genre_counts):
    """
    Generate executive summary of data assets
    """
    
    print(f"\n" + "="*60)
    print("📋 EXECUTIVE DATA SUMMARY")
    print("="*60)
    
    # Dataset overview
    print(f"🎬 MOVIE DATABASE OVERVIEW:")
    print(f"  Total movies: {len(business_dataset):,}")
    print(f"  Time period: {business_dataset['startYear'].min():.0f} - {business_dataset['startYear'].max():.0f}")
    print(f"  Average rating: {business_dataset['averageRating'].mean():.2f}/10")
    print(f"  Total user votes: {business_dataset['numVotes'].sum():,}")
    
    # Business-relevant segments
    print(f"\n📊 BUSINESS-RELEVANT SEGMENTS:")
    recent_high_cred = business_dataset[
        business_dataset['recent_movie'] & business_dataset['high_credibility']
    ]
    print(f"  Recent + High Credibility: {len(recent_high_cred):,} movies")
    print(f"  AAA Analysis Ready: {len(recent_high_cred):,} movies")
    
    # 2020 specific (for business questions)
    movies_2020 = business_dataset[business_dataset['startYear'] == 2020]
    print(f"  2020 movies: {len(movies_2020):,}")
    print(f"  2020 with high credibility: {len(movies_2020[movies_2020['high_credibility']]):,}")
    
    # Genre diversity
    print(f"\n🎭 CONTENT DIVERSITY:")
    print(f"  Unique genres: {len(genre_counts)}")
    print(f"  Most common: {genre_counts.head(3).index.tolist()}")
    
    # Data quality score
    completeness_score = (
        (business_dataset['averageRating'].notna().sum() / len(business_dataset)) * 0.3 +
        (business_dataset['numVotes'].notna().sum() / len(business_dataset)) * 0.3 +
        (business_dataset['genres'].notna().sum() / len(business_dataset)) * 0.2 +
        (business_dataset['runtimeMinutes'].notna().sum() / len(business_dataset)) * 0.2
    ) * 100
    
    print(f"\n✅ DATA QUALITY SCORE: {completeness_score:.1f}/100")
    print(f"   Ready for advanced analytics: {'YES' if completeness_score > 80 else 'NEEDS IMPROVEMENT'}")
    
    # Recommendations
    print(f"\n💡 RECOMMENDATIONS:")
    if len(recent_high_cred) >= 1000:
        print("  ✅ Sufficient data for robust AAA analysis")
    else:
        print("  ⚠️ Consider expanding time window for larger sample size")
    
    if completeness_score > 85:
        print("  ✅ High data quality - proceed with confidence")
    else:
        print("  ⚠️ Address missing data before critical analysis")
    
    print("  📈 Ready for EDA and business question analysis")
    
    return {
        'total_movies': len(business_dataset),
        'recent_high_cred': len(recent_high_cred),
        'movies_2020': len(movies_2020),
        'data_quality_score': completeness_score,
        'genre_diversity': len(genre_counts)
    }

In [ ]:
data_summary = generate_data_summary(business_ready_data, genre_counts)